In [1]:
import json 
import pandas as pd
from urllib.request import urlopen

'''
get_jsonparsed_data is copied directly from https://financialmodelingprep.com.
This is the main call to process the variable url's to the API
'''
def get_jsonparsed_data(url):
    """
    Receive the content of ``url``, parse it as JSON and return the object.

    Parameters
    ----------
    url : str

    Returns
    -------
    dict
    """
    response = urlopen(url)
    data = response.read().decode("utf-8")
    return json.loads(data)

'''
financialmodelingprep ask that the API key not be shared.  I keep mine in a json
file.  The API key is available and free for use provided by registration at 
the site, it is to not be shared or left in git. 
'''
api_key = None
with open('../../.api_keys/api_keys.json') as f:
  api_keys = json.load(f)
api_key = api_keys['fiancialmod']

# Retrieving profile data (today's metrics) stocks

In [2]:
symbol = 'PYPL'
prefix = 'https://financialmodelingprep.com/api/v3/profile/'

# Note url format for profile data for stock of symbol
url = '{}{}?apikey={}'.format(prefix,symbol, api_key)
url_noAPI = '{}{}?apikey={}'.format(prefix,symbol, 'YourAPIKeyHere')

print('constructed url: \n\n{}\n'.format(url_noAPI))

# retrieve profile data
profile_result = get_jsonparsed_data(url)

# profile_result is a single item list.
profile_result = profile_result[0]

# actual data is in a json
print('result from profile url are today metrics')
print('result is a json of these key: value pairs\n')
for key in profile_result.keys():
    print('\t {}: {}'.format(key,profile_result[key]))

constructed url: 

https://financialmodelingprep.com/api/v3/profile/PYPL?apikey=YourAPIKeyHere

result from profile url are today metrics
result is a json of these key: value pairs

	 symbol: PYPL
	 price: 286.92
	 beta: 1.07513
	 volAvg: 8786646
	 mktCap: 336034988000
	 lastDiv: 0.0
	 range: 82.07-309.14
	 changes: -3.89
	 companyName: PayPal Holdings Inc
	 currency: USD
	 cik: 0001633917
	 isin: US70450Y1038
	 cusip: 70450Y103
	 exchange: Nasdaq Global Select
	 exchangeShortName: NASDAQ
	 industry: Credit Services
	 website: https://www.paypal.com/
	 description: PayPal Holdings, Inc. operates as a technology platform and digital payments company that enables digital and mobile payments on behalf of consumers and merchants worldwide. Its payment solutions include PayPal, PayPal Credit, Braintree, Venmo, Xoom, and iZettle products. The company's payments platform allows consumers to send and receive payments, withdraw funds to their bank accounts, and hold balances in their PayPal acc

In [3]:
mktCap = profile_result['mktCap']
share_price = profile_result['price']
total_shares = int( mktCap/share_price )
'Approximate number of shares {:,}'.format( total_shares )

'Approximate number of shares 1,171,180,078'

# Historical data (daily metrics) stocks

In [4]:

prefix = 'https://financialmodelingprep.com/api/v3/historical-price-full/'
url = '{}{}?apikey={}'.format(prefix, symbol, api_key)
historic_result = get_jsonparsed_data(url)

# historical_result is a dictionary to two key:values 'symbol' and 'historical'
# symbol is just the ticker symbol
# historical is a list of jsons, one per trading day, going back 1259 ~ 4 years

day_records = historic_result['historical']

In [5]:
# each record has the following keys
day_records[0] 

{'date': '2021-02-19',
 'open': 292.119995,
 'high': 293.940002,
 'low': 285.459991,
 'close': 286.920013,
 'adjClose': 286.920013,
 'volume': 9397600.0,
 'unadjustedVolume': 9397600.0,
 'change': -5.19998,
 'changePercent': -1.78,
 'vwap': 288.77334,
 'label': 'February 19, 21',
 'changeOverTime': -0.0178}

# Qarterly data (metrics) stocks

In [6]:
prefix = 'https://financialmodelingprep.com/api/v3/income-statement/'
url = '{}{}?period=quarter&limit=400&apikey={}'.format(prefix,symbol, api_key)
quarterly_result = get_jsonparsed_data(url)

In [7]:
quarterly_result[0:3]

[{'date': '2020-12-31',
  'symbol': 'PYPL',
  'reportedCurrency': 'USD',
  'fillingDate': '2021-02-05',
  'acceptedDate': '2021-02-04 19:30:14',
  'period': 'Q4',
  'revenue': 6116000000,
  'costOfRevenue': -7378000000,
  'grossProfit': 13494000000,
  'grossProfitRatio': 2.2063440156965335,
  'researchAndDevelopmentExpenses': -1236000000,
  'generalAndAdministrativeExpenses': -3469000000,
  'sellingAndMarketingExpenses': 605000000,
  'otherExpenses': 14249000,
  'operatingExpenses': 12626000,
  'costAndExpenses': 5248000000,
  'interestExpense': -150000000,
  'depreciationAndAmortization': 301000000,
  'ebitda': 1169000000,
  'ebitdaratio': 0.19113799869195552,
  'operatingIncome': 868000000,
  'operatingIncomeRatio': 0.1419228253760628,
  'totalOtherIncomeExpensesNet': 913000000,
  'incomeBeforeTax': 1859000000,
  'incomeBeforeTaxRatio': 0.3039568345323741,
  'incomeTaxExpense': 292000000,
  'netIncome': 1567000000,
  'netIncomeRatio': 0.25621321124918245,
  'eps': 1.3370307167235496,

In [8]:
Q2021_4 = quarterly_result[0]

revenue_per_share = Q2021_4['revenue']/total_shares
print(revenue_per_share)

grossProfit_per_share = Q2021_4['grossProfit']/total_shares
print(grossProfit_per_share)

PE = share_price/Q2021_4['eps']
print(PE)

5.222083362657745
11.521712376668347
214.59492022973836


In [9]:
# MSFT
6.71/2.05

3.2731707317073173

In [10]:
#Paypal
3.54/1.3370307167235496

2.64765794511806

In [11]:
prefix = 'https://financialmodelingprep.com/api/v3/ratios/'
url = '{}{}?period=quarter&limit=400&apikey={}'.format(prefix,symbol, api_key)
ratios_result = get_jsonparsed_data(url)

In [12]:
ratios_result

[{'symbol': 'PYPL',
  'date': '2020-12-31',
  'currentRatio': 1.3263713683772467,
  'quickRatio': 0.3552943012458709,
  'cashRatio': 0.12469113324836788,
  'daysOfSalesOutstanding': 34.43508829300196,
  'daysOfInventoryOutstanding': 0,
  'operatingCycle': 8.490843688685416,
  'daysOfPayablesOutstanding': -3.074003795066414,
  'cashConversionCycle': 11.56484748375183,
  'grossProfitMargin': 2.2063440156965335,
  'operatingProfitMargin': 0.1419228253760628,
  'pretaxProfitMargin': 0.3039568345323741,
  'netProfitMargin': 0.25621321124918245,
  'effectiveTaxRate': 0.157073695535234,
  'returnOnAssets': 0.02226516432458546,
  'returnOnEquity': 0.07810397248666699,
  'returnOnCapitalEmployed': 0.05821746210697733,
  'netIncomePerEBT': 0.842926304464766,
  'ebtPerEbit': 1,
  'ebitPerRevenue': 0.3039568345323741,
  'debtRatio': 0.7149291692124071,
  'debtEquityRatio': 2.5079001146388875,
  'longTermDebtToCapitalization': 0.30822012275015515,
  'totalDebtToCapitalization': None,
  'interestCov

In [23]:
prefix = 'https://financialmodelingprep.com/api/v3/ratios/'
url = '{}{}?period=year&limit=2&apikey={}'.format(prefix,symbol, api_key)
ratios_result = None
ratios_result = get_jsonparsed_data(url)

In [24]:
ratios_result

[{'symbol': 'PYPL',
  'date': '2020-12-31',
  'currentRatio': 1.3263713683772467,
  'quickRatio': 0.3552943012458709,
  'cashRatio': 0.12469113324836788,
  'daysOfSalesOutstanding': 9.816584319940338,
  'daysOfInventoryOutstanding': None,
  'operatingCycle': None,
  'daysOfPayablesOutstanding': None,
  'cashConversionCycle': None,
  'grossProfitMargin': 1.0,
  'operatingProfitMargin': 0.15330474503589075,
  'pretaxProfitMargin': 0.2360865106740002,
  'netProfitMargin': 0.19586091171809453,
  'effectiveTaxRate': 0.17038499506416585,
  'returnOnAssets': 0.05970530982253229,
  'returnOnEquity': 0.2094402631710113,
  'returnOnCapitalEmployed': 0.1586183139170738,
  'netIncomePerEBT': 0.8296150049358342,
  'ebtPerEbit': 1.0,
  'ebitPerRevenue': 0.2360865106740002,
  'debtRatio': 0.7149291692124071,
  'debtEquityRatio': 2.5079001146388875,
  'longTermDebtToCapitalization': 0.30822012275015515,
  'totalDebtToCapitalization': None,
  'interestCoverage': None,
  'cashFlowToDebtRatio': None,
  '

In [33]:
prefix = 'https://financialmodelingprep.com/api/v3/ratios/'
symbol = 'SQM'
url = '{}{}?period=quarter&limit=400&apikey={}'.format(prefix,symbol, api_key)
ratios_result = get_jsonparsed_data(url)

In [34]:
ratios_result

[{'symbol': 'SQM',
  'date': '2020-09-30',
  'currentRatio': 3.657196894127411,
  'quickRatio': 2.1249039057491625,
  'cashRatio': 0.8479449795575276,
  'daysOfSalesOutstanding': 441.09425579741827,
  'daysOfInventoryOutstanding': 290.8055142307021,
  'operatingCycle': 399.5684814136272,
  'daysOfPayablesOutstanding': 46.84880821751694,
  'cashConversionCycle': 352.71967319611025,
  'grossProfitMargin': 0.25341927335196984,
  'operatingProfitMargin': 0.05076467185133172,
  'pretaxProfitMargin': 0.0213017307242191,
  'netProfitMargin': 0.0037604278452726364,
  'effectiveTaxRate': 0.6539856950347258,
  'returnOnAssets': 0.0003543770455234043,
  'returnOnEquity': 0.0008317009431967302,
  'returnOnCapitalEmployed': 0.00237977181621955,
  'netIncomePerEBT': 0.17653156421685498,
  'ebtPerEbit': 1,
  'ebitPerRevenue': 0.0213017307242191,
  'debtRatio': 0.5646772442908464,
  'debtEquityRatio': 1.3252624644036393,
  'longTermDebtToCapitalization': 0.44971752353791355,
  'totalDebtToCapitalizati

In [ ]:
def get_current_metrics(symbol, api_key):
    prefix = 'https://financialmodelingprep.com/api/v3/profile/'
    url = '{}{}?apikey={}'.format(prefix,symbol, api_key)
    result = get_jsonparsed_data(url)
    return result

def get_historic_data(symbol, api_key):
    def extract_history(result):
        datestamp =[]
        close = []
        high = []
        low = []
        volume = []
        for token in result['historical']:
            datestamp.append(token['date'])
            close.append(token['close'])
            high.append(token['high'])
            low.append(token['low'])
            volume.append(token['volume'])
        df = pd.DataFrame.from_dict({ 'symbol':symbol, 'datestamp':datestamp, 'close':close,
                                       'high':high, 'low':low, 'volume':volume})
        return df
    
    prefix = 'https://financialmodelingprep.com/api/v3/historical-price-full/'
    url = '{}{}?apikey={}'.format(prefix, symbol, api_key)
    result = get_jsonparsed_data(url)
    return extract_history(result)

symbol = 'PYPL'

df = get_historic_data(symbol, api_key)

In [ ]:
df.head(10)

In [14]:
ratios_result

{'Error Message': 'Invalid API KEY. Please retry or visit our documentation to create one FREE https://financialmodelingprep.com/developer/docs'}

In [ ]:
symbol = 'PYPL'

def get_quarterly_metrics(symbol, api_key):
    def extract_history(result):
        start_date =[]
        revenue = []
        grossProfit = []
        for token in result:
            start_date.append(token['date'])
            revenue.append(token['revenue'])
            grossProfit.append(token['grossProfit'])
        df = pd.DataFrame.from_dict({ 'symbol':symbol, 'start_date':start_date, 'revenue':revenue,
                                      'grossProfit':grossProfit})
        end_date = list( df['start_date'] )
        end_date = end_date[:-1]
        end_date.insert(0,None)
        df['end_date'] = end_date
        fields = ['symbol','start_date','end_date','revenue','grossProfit']
        return df[fields]

    prefix = 'https://financialmodelingprep.com/api/v3/income-statement/'
    url = '{}{}?period=quarter&limit=400&apikey={}'.format(prefix,symbol, api_key)
    result = get_jsonparsed_data(url)
    return extract_history(result)

current_metrics = get_quarterly_metrics(symbol, api_key)

current_metrics.head(20)

In [ ]:
end_date = [0,1,2,3]
end_date = end_date[:-1]
end_date.insert(0,None)
print( end_date )

In [ ]:
symbol = 'ETHUSD'
df = get_historic_data(symbol, api_key)
df.head(10)

In [ ]:
symbol = 'SQ'
prefix = 'https://financialmodelingprep.com/api/v3/historical-price-full/'
url = '{}{}?apikey={}'.format(prefix,symbol, api_key)

print(url)

result = get_jsonparsed_data(url)

len(result['historical'])

In [ ]:
symbol = 'ETHUSD'
prefix = 'https://financialmodelingprep.com/api/v3/historical-price-full/'
url = '{}{}?apikey={}'.format(prefix,symbol, api_key)

print(url)

result = get_jsonparsed_data(url)

len(result['historical'])

In [ ]:
result['historical']